In [1]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np 
import sys

In [173]:
class Node():
    def __init__(self, name=None, new_feature=None, c=None, is_leaf=None, ans=None):
        self.name = name
        self.new_feature = new_feature
        self.c = c
        self.is_leaf = is_leaf
        self.ans = ans       
    def __str__(self):
        st = "Node name:  " + str(self.name)
        st += ", feature name: " + str(self.new_feature)
        st += ", c: "  + str(self.c)
        st += ", is leaf: " + str(self.is_leaf) 
        st += ", predicted answer is: " + str(self.ans)
        return st
        
class SimpleBinaryDicisionTree():
    def __init__(self, criterion='gini',max_depth=None):
        """
        A decision tree classifier.
        Class marks are {+1, -1}
        
        criterion : string, optional (default="gini", also support "entropy")
        The function to measure the quality of a split.
        
        max_depth : int or None, optional (default=None)
        The maximum depth of the tree. 
        
        """
        self.criterion = criterion
        self.max_depth = max_depth
        self.depth = 0
        self.tree = [] 
    
    def p(self, X, class_mark):
        """proportion of objects of class mark class"""
        
        n = float(len(X))
        if (n == 0):
            return float(sys.maxint)
        return float(len([i for i in range(len(X)) if X[i] == class_mark]))
    
    def C(self,y, class_mark):
        """classification error criterion"""
        return 1 - self.p(y, class_mark)
        
        
    def H(self,y, class_mark):
        """entropy criterion"""
        p = self.p(y,class_mark)
        return - p*np.log(p) - (1 - p)*np.log(1 - p)
        
    def F(self,y, class_mark = -1):
        """gini criterion"""
        p = self.p(y,class_mark)
        return 1 - (p**2 + (1 - p)**2) 

        
    def crit(self,y, y_l, y_r, class_mark = -1, criterion = 'gini'):
        """general criterium"""
        f = self.C
        if (criterion == 'gini'):
            f = self.F
        elif (criterion == 'entropy'):
            f = self.H
        elif (criterion == 'simple'):
            f = self.C
        n = float(len(y))
        n_l = float(len(y_l))
        n_r = float(len(y_r))
        if (n_r == 0) or (n_l == 0) or (n == 0):
            return float(sys.maxint)
        return f(y, class_mark) - n_l / n * f(y_l, class_mark) - n_r / n * f(y_r, class_mark)
    
    def split(self,X_data, y_data, criterion):
        min_cr =  sys.maxint
        min_feature = None
        min_split_val = None
        for i in range(X_data.shape[1]):
            for c in np.unique(X_data):
                y_l = [y_data[j] for j in range(len(y_data)) if (X_data[j][i]<=c)]
                y_r = [y_data[j] for j in range(len(y_data)) if (X_data[j][i]>c)]
                cr = self.crit(y_data, y_l,y_r, -1,criterion)
                if cr < min_cr:
                    min_cr = cr
                    min_feature = i
                    min_split_val = c
        return min_feature, min_split_val
    
    def fit(self, X_data, y_data):
        self.tree = []
        x_node = np.zeros(len(X_data))
        if (self.max_depth == None):
            self.max_depth = X_data.shape[1] // 2
        for depth in range(self.max_depth + 1):
            for n_in_row in range(2**depth):
                name = 2**(depth) + n_in_row - 1 
                indexes_in_node = np.arange(len(X_data))
                if (depth != 0):
                    indexes_in_node = [j for j in range(len(x_node)) if (x_node[j] == name)] 
                Y_new = y_data[indexes_in_node]
                X_new = X_data[indexes_in_node]
                
                feature, c = self.split(X_new, Y_new, self.criterion)
                
                is_leaf = True
                ans = None
                
                if (depth == self.max_depth) or (feature == None):
                    is_leaf = True
                    if (np.sum(y_data[indexes_in_node])< 0):
                        ans = -1
                    else:
                        ans = 1    
                else:
                    is_leaf = False
                    for j in indexes_in_node:
                        if (X_data[j][feature] <= c):
                            x_node[j] = 2 * (name + 1) - 1
                        else:
                            x_node[j] = 2 * (name + 1)
                me = Node(name, feature, c, is_leaf, ans)
                self.tree.append(me)
    
    def predict(self, X_data):
        y = np.zeros(len(X_data))
        now = 0
        for i in range(len(X_data)):
            while not ((now >= len(self.tree)) or (self.tree[now].is_leaf)):
                if (X_data[i][self.tree[now].new_feature] <= self.tree[now].c):
                    now = 2 * (now + 1) - 1
                else:
                    now = 2 * (now + 1)
            y[i] = self.tree[now].ans
        return y
    
    def __str__(self):
        st = ""
        for node in self.tree:
             st += str(node) + "\n"
        return st

In [3]:
boston = load_boston()
print boston.DESCR

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

In [4]:
boston_data = pd.DataFrame(boston.data)
boston_data.columns = boston.feature_names
boston_data['target'] = boston.target
boston_data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [5]:
#Так как я сделала дерево для задачи классификации(до последнего не знала, что это, 
#оказывается, вполне себе регрессия, то буду делать на данных  второго задания)

In [174]:
frame = pd.read_csv('german_credit_data/german_credit.csv', header=0, sep=',')

model = SimpleBinaryDicisionTree(max_depth=2,criterion='gini')
target = (frame.Creditability * 2  - 1).values[:100]

model.fit(frame.values[:100,:10], target)
print model

Node name:  0, feature name: 0, c: 0, is leaf: False, predicted answer is: None
Node name:  1, feature name: 2, c: 18, is leaf: False, predicted answer is: None
Node name:  2, feature name: 1, c: 1, is leaf: False, predicted answer is: None
Node name:  3, feature name: 1, c: 1, is leaf: True, predicted answer is: -1
Node name:  4, feature name: 3, c: 2, is leaf: True, predicted answer is: -1
Node name:  5, feature name: 2, c: 6, is leaf: True, predicted answer is: 1
Node name:  6, feature name: 1, c: 2, is leaf: True, predicted answer is: 1



In [172]:
print model.predict(frame.values[:, :10])

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1

К сожалению, для данных German Credit Data для того, чтобы дерево не проявило свои колоссальные способности к переобучению, нужно использовать более чуствительные критерии останова, нежели использованые в данном примере